# Intro to SCS API
This Notebook makes simple queries against the Stratus Cloud Services Asset and Index APIs. 
To do this it needs to authenticate.
It also demonstrates how to get poster frames for assets.
Hello Casiano.

In [ ]:
#BaseUrl is the address of the load balancer entry point to the SCS instance
baseUrl = 'https://scs-newbury.gvnewsdev.com/'

# These paths are the access points for key APIs in SCS
identityPath = 'identity/'
assetPath = 'scs-db/asset-metadata/'
indexPath = 'scs-index/asset/metadata/'
stillsPath = 'scs-store/still/'

# Add these to the base paths to find the Swagger UI and documentation
swaggerUI = 'swagger/index.html'
swaggerPath = 'swagger/v1/swagger.json'

# We always need a client access key to call SCS APIs
# The client key is associated with an account - this Account determines what data we can see in the backend microservices.
# This account based data access is called 'multi-tennancy' in the literature. 
# See here for info: https://openid.net/connect/
newbury_key = 'MTExMjhhMDc5ZmNiNDdkMTljYjUyOGY3N2RiNDEwNWY6SXpZYWxnN1BndVovaGt3eUZpNUdPWUEzdkhtNnVYWkUrN2F1TDBYTE84SVBYa2VMQWxsbHd1VDU3MFhHU3ZPV0dJYi9NMXI5RUlZTjhoWmRNNlJWb1E9PQ=='


In [ ]:
# Get a JWT Bearer token from our client key.

import requests # for http get & post etc
import json # for json support

headers = {"Authorization": "Basic " + newbury_key,
          "Content-Type": "application/x-www-form-urlencoded"}
data = {"grant_type": "client_credentials","scope": "platform"}

response = requests.post(baseUrl + identityPath + 'connect/token', headers=headers, data=data)
print(response.url)
print(response)
print()

newbury_token = response.json()["access_token"]

# You can use the Bearer token printed below to authorise the swagger API pages.
print('Bearer ' + newbury_token)

# Surf here: https://jwt.io/ to decode the Bearer token ...
print()
print('Use the Bearer token above to authorise the Swagger UIs when you try them.')
print('Visit https://jwt.io/ to analayse the JWT you have just made.')
print()

# Add a link to the documentation for this API
docPath = baseUrl + identityPath + swaggerUI
print(docPath)
result = requests.get(baseUrl + indexPath + swaggerPath)
print(result.url)
print(result)

In [ ]:
# Make a simple (blank) index search to find some assets

headers = {"Authorization": "Bearer " + newbury_token,
          "Content-Type": "application/json-patch+json"}
data = '{"search": ""}' # blank query here just lists all assets - add an asset title here to search for assets

path = baseUrl + indexPath + 'query'
print(path)
response = requests.post(path, headers=headers, data=data)
print(response)
# print(json.dumps(response.json(), indent=2))

j = response.json()
assets = {}
assetIDs = []
clips = {}
clipIDs = []
for i in j["results"]:
    asset = i['asset']
    
    # The asset id gives access to the whole metadata entry. 
    # The asset ID is a key used in the asset & other metadata stores.
    assetid = asset['asset:id']
    assetIDs.append(assetid)
    assets[assetid] = asset
    
    # The clip ID is the identity of the part of the asset to do with TV.
    # The clip ID is a key used in the stills, streams & other essence stores.
    clipid = asset['version']["clip:id"] 
    clipIDs.append(clipid)
    clips[clipid] = asset
    
print('Num entries: ' + str(len(clips)))

# Add a link to the documentation for this API
docPath = baseUrl + indexPath + swaggerUI
print(docPath)
result = requests.get(baseUrl + indexPath + swaggerPath)
print(result.url)
print(result)

In [ ]:
# Enum some clips, with asset metadata and poster frames (where available)

from IPython.core.display import Image, display, DisplayObject

headers = {"Authorization": "Bearer " + newbury_token}

for id in clipIDs[0:15]: # limit to 1st few entries for demo ...
    print(id)
    print(json.dumps(clips[id], indent=2))
    
    path = baseUrl + stillsPath + id + '/poster.jpeg'
    #print(path)
    
    response = requests.get(path, headers=headers)
    #print(response)
    
    if 200 == response.status_code:
        print(response.url)
        display(Image(data=response.content,format=u'jpg'))
    else:
        print('No Poster available for this clip')

# Add a link to the documentation for this API
docPath = baseUrl + stillsPath + swaggerUI
print(docPath)
result = requests.get(baseUrl + stillsPath + swaggerPath)
print(result.url)
print(result)

In [ ]:
# Access asset metadata given only an asset ID.

headers = {"Authorization": "Bearer " + newbury_token}

# First lets get the schema for the assets ...
path = path = baseUrl + assetPath  + 'schema'
response = requests.get(path, headers=headers)
print(response.url)
print(response)
print(json.dumps(response.json(), indent=2))

# Now lets get a few assets
for id in assetIDs[0:5]: # limit to 1st few entries for demo ...
    print(id)
    #print(assets[id])
    
    path = path = baseUrl + assetPath  + 'asset/' + id
    response = requests.get(path, headers=headers)
    print(response.url)
    print(response)
    print(json.dumps(response.json(), indent=2))

# Add a link to the documentation for this API
docPath = baseUrl + assetPath + swaggerUI
print(docPath)
result = requests.get(baseUrl + assetPath + swaggerPath)
print(result.url)
print(result)    